# 10-2 Mnist cnn 정리

- 전체적인 프로세스
<t>
1) import <br>
2) Device 설정하기 (cuda로 하게 되면, cuda 전용 Tensor형태로 변환시켜야 해서, to(device) 구문 사용함) <br>
3) Dataset 불러오기 (이때, import torch.transforms 사용해서, 데이터를 Tensor로 바꿈) <br>
4) Dataloader 만들기 (이떄, from torch.utils.data import DataLoader) <br>
5) Model 구현하기 -> Class 형태로 만들면 됌 <br>
6) Loss & optimizer 정하기 <br>
7) Training <br>
8) Test <br>(with torch.no_grad(), model.eval 모드를 사용해야 dropout이나 Batchnorm 과정에서 train에서 학습된 내용을 사용함) <br>
<br>
- 헷갈리는 코드  <br>
import torchvision.datasets as dsets <br>
import torchvision.transforms as transforms <br>
    
<h6>
device = "cuda" if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == "cuda": <br>
<t>    torch.cuda.manual_seed_all(777) <br>
print(device)  
</h6>
  
<br>
- 디버깅 <br>
가령, CNN에서 FC로 넘어갈 때 (batch_size, 나머지) 크기로 Data의 Rank가 변환된다. <br>
이때, forward과정에서 print(out.size())와 같은 구문을 통해서 디버깅을 하면 효과적으로 제어할 수 있다. <br>
    <br>
 또한, 모델 테스팅 할때 torch.Tensor(batch,Ic,IH,IW) 형태로 하나 넣어놓고 잘 돌아가는지 확인하는 방법도 존재한다.


# 10-3 Visdom 정리

다운로드: pip install visdom <br>
서버열기: python -m visdom.server

- 사용이유: Loss function의 변동과정을 서버상에서 보여주기 위함이다.
- 사용방법: 아래와 같은 함수를 선언하고, train 과정에서 epoch가 끝날 때마다, 해당 구문을 추가하면 된다.

In [2]:
import visdom
import torch

vis = visdom.Visdom()

def loss_tracker(loss_plot, loss_value, num):
    vis.line(X = num,
             Y = loss_value,
             win = loss_plot,
             update = "append"
            )
    
plt = vis.line(Y = torch.Tensor(1).zero_())

loss_tracker(plt, cost.item(), epoch)

Setting up a new session...


NameError: name 'cost' is not defined

# 10-4 ImageFolder 사용하기
- 내가 가지고 있는 Dataset을 통해 학습을 진행하는 과정을 배운다.
- 내가 이 내용을 딥러닝 응용 수업 때 배웠더라면, 아마 많은 에로사항들이 해결되었을 터인데..


In [ ]:
# 데이터를 불러와서, Resize시키고, 다시 저장시키는 방법

#1) Resize
import torchvision
import torchvision.transforms as transforms
trans = transforms.Compose([
    transforms.Resize((64,128)),
    #transforms.ToTensor(), --> 여기서는 데이터를 Resize해서 저장할 예정이라 굳이 사용하지 않았음.
    ])

# Image 폴더는, train_data(빔), class_1, class_2 폴더로 구성되어 있음.
train_data = torchvision.datasets.ImageFolder(root = "Image",
                                             transform = trans)

In [ ]:
# 데이터를 이렇게 불러오면 됌 -> 폴더마다 라벨링이 다르다고 인식함.
for num,sample in enumerate(train_data):
    imgs, label = sample
    print(num, imgs, label)
    
    #여기서 Resize된 데이터를 미리저장 해놓으면 됌.
    if label == 0:
        imgs.save("Image/train_data/class_1/%d_%d.jpeg"%(num,label))
    else:
        imgs.save("Image/train_data/class_2/%d_%d.jpeg"%(num,label))
    

In [ ]:
# 다시 불러오기.
train_data = torchvision.datasets.ImageFolder(root = "Image/train_data",
                                              transform= transforms.ToTensor())
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_data,
                          batch_size= 8,
                          shuffle= True,
                          drop_last= False,
                         )

# Weight 저장 및 불러오기

In [ ]:
# 저장 (여기서 모델은 이미 구현된 class 형태의 모델을 의미함)
torch.save(model.state_dict(), "model/model.pth")

# 불러오기
new_net = CNN().to(device)
new_net.load_state_dict(torch.load("./model/model.pt"))


### 다음시간: HW5 다시 해봐야겠다.